### Install the needed Libraries

In [ ]:
! pip install opencv-python --user
! pip install numpy --user
! pip install matplotlib --user
! pip install pillow --user
! pip install matplotlib --user
! pip install scikit-image --user
! pip install sklearn --user
! pip install python-resize-image --user

### Upload the training Dataset and Preprocess the Data

In [ ]:
import cv2
import glob
from PIL import Image
from resizeimage import resizeimage
import numpy as np

# Read RGB images from the training folder
# Note if the code is runing on the local PC don not upload all the DATASET upload the apropriate number ONLY!!!
# Which is usaally between 2000 to 3000 images !! to prevent PC crash due to RAM Memory Loss !!!!!!!!
# To control the Number of Images go to the training Folder and add or delet the files as appropriate
females = [cv2.imread(file) for file in glob.glob('Training\\female\\*.jpg')]
males = [cv2.imread(file) for file in glob.glob('Training\\male\\*.jpg')]
print("Done Uploading the Images")

#convert the images to greyscale
f_gray= [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in females]
m_gray=[cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in males]
print("Done Converting the Images to Gray")

#histogram equalization
f_equlized = [cv2.equalizeHist(img) for img in f_gray]
m_equlized = [cv2.equalizeHist(img) for img in m_gray]
print("Apllied Histogram Equalization on the Gray Images")

#resizing
size = [90,90]
f_resized = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in f_equlized]
m_resized = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in m_equlized]
print("Done Resizing the Histogram Equalized Images to 90*90 Dimensions")

#Convert the resized arrays to numpy array to be used by the Haar Cascade !!
f_sized = [np.array(img) for img in f_resized]
m_sized = [np.array(img) for img in m_resized]


# to view any of the images Whether colerd or Preprocessed Uncomment the next Lines !!!!
# AND Change the variable Named (Array_Name) to any of the preferred arrays above !!!!!!
#for img in Array_Name:
    # Output img with window name as 'image' 
    #cv2.imshow('image',img)
    # Maintain output window utill 
    # user presses a key 
    #cv2.waitKey(0)
    # Destroying present windows on screen 
    #cv2.destroyAllWindows()
    



###  Apply the Haar Cascade on the Preprocessed Images

In [63]:
#Upload the Haar Cascade Model for frontal Face Detection 
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Detect the Faces in the female images
f_haar= [face_cascade.detectMultiScale(
    img,
    scaleFactor=1.1,
    minNeighbors=3,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    ) for img in f_sized]
    
print("Done Detecting the Female Faces")

# Detect the Faces in the male images
m_haar=[face_cascade.detectMultiScale(
    img,
    scaleFactor=1.1,
    minNeighbors=3,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
) for img in m_sized]
print("Done Detecting the Male Faces")

Done Detecting the Female Faces
Done Detecting the Male Faces


### Extracting the Region of Interest (roi) Detect by the Haar Cascade

In [65]:
# print The Number Of Detected Faces
print ("Found {0} faces!".format(len(f_haar) + len(m_haar)))

# Extract the region of interest in the image (roi)
for f in f_haar:
        for (x, y, w, h) in f:
            f_roi = [img[y:y+h+88, x:x+w+88] for img in f_sized]
            
print("Extracted the region of interest for the female faces")      

for m in m_haar:
        for (x, y, w, h) in m:
            m_roi = [img[y:y+h+88, x:x+w+88] for img in m_sized]
            
print("Extracted the region of interest for the male faces")

Found 2006 faces!
Extracted the region of interest for the female faces
Extracted the region of interest for the male faces


### To See the region of interest Extracted by the Haar Cascade

In [ ]:
for img in f_roi:            
    cv2.imshow("Faces found", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
for img in m_roi:            
    cv2.imshow("Faces found", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Resizing the region of interest for larger Dimensions to Extract More Accurate Featers By the HOG and LBP

In [68]:
#resizing
size = [320,320]

f_resized2 = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in f_roi]
print("Done Resizing the female faces region of interest to 320*320 Dimensions")

m_resized2 = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in m_roi]
print("Done Resizing the male faces region of interest to 320*320 Dimensions")

#Convert the resized arrays to numpy array to be used by the HOG and LBP !!
f_roi2 = [np.array(img) for img in f_resized2]
m_roi2 = [np.array(img) for img in m_resized2]

Done Resizing the female faces region of interest to 320*320 Dimensions
Done Resizing the male faces region of interest to 320*320 Dimensions


### Defining The Function of the Local Binary Patterns (LBPs)

In [51]:
from skimage import feature
import numpy as np

def describe(image, numPoints, radius, eps=1e-7):
    # compute the Local Binary Pattern representation
    # of the image, and then use the LBP representation
    # to build the histogram of patterns
    lbp = feature.local_binary_pattern(image, numPoints, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
    bins=np.arange(0, numPoints + 3),
    range=(0, numPoints + 2))
    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    # return the histogram of Local Binary Patterns
    return hist

### Using HOG and LBP to Extract the features from the Resized Images !!!

In [ ]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from skimage.feature import local_binary_pattern
from scipy import sparse

data= [] # Array for storing the combined Feature Vectors of the HOG and LBP
labels = [] # Array for storing the Labels of the combined Feature Vectors of the HOG and LBP 

# settings for HOG
cs = (3, 3) #Cell Size
orn = 8     #Number of Orientations

# settings for LBP
radius = 3
n_points = 8 * radius

#Extracting the features of the Female Faces
for img in f_roi2:    
    #Extract the HOG Feature Vector
    fd = hog(img, orientations=orn, pixels_per_cell=cs, cells_per_block=(1, 1), block_norm='L2', feature_vector=True, multichannel=False)
    #Extract the LBP Feature Vector
    hist = describe(img, n_points, radius)
    #Combine Both Features into One Vector
    feat = np.hstack([hist, fd])
    #add the vector to the Data Array
    data.append(feat)
    # add the label of the feature in this case 1 for a female Feature
    labels.append(1)

print("DONE Extracting the features of the Female Faces")

#Extracting the features of the Male Faces
for img in m_roi2: 
    #Extract the HOG Feature Vector
    fd = hog(img, orientations=orn, pixels_per_cell=cs, cells_per_block=(1, 1), block_norm='L2', feature_vector=True, multichannel=False)
    #Extract the LBP Feature Vector
    hist = describe(img, n_points, radius)
    #Combine Both Features into One Vector
    feat = np.hstack([hist, fd])
    #add the vector to the Data Array
    data.append(feat)
    # add the label of the feature in this case 1 for a female Feature
    labels.append(0)

print("DONE Extracting the features of the Female Faces")

### HOG Visualization

In [ ]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

i = 0

# settings for HOG
cs = (3, 3) #Cell Size
orn = 8     #Number of Orientations

while i < 1:
    i = i+1
    fd, hog_image = hog(females[i], orientations=orn, pixels_per_cell=cs,
                        cells_per_block=(1, 1), visualize=True, multichannel=True)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(females[i], cmap=plt.cm.gray)
    ax1.set_title('Input image')

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    
i = 0
while i < 1:
    i = i+1
    
    fd, hog_image = hog(f_gray[i], orientations=orn, pixels_per_cell=cs,
                        cells_per_block=(1, 1), visualize=True, multichannel=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(f_gray[i], cmap=plt.cm.gray)
    ax1.set_title('Gray image')

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    
i = 0
while i < 1:
    i = i+1
    
    fd, hog_image = hog(f_resized2[i], orientations=orn, pixels_per_cell=cs,
                        cells_per_block=(1, 1), visualize=True, multichannel=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(f_resized2[i], cmap=plt.cm.gray)
    ax1.set_title('Resized image')

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()
    
i = 0
while i < 1:
    i = i+1
    
    fd, hog_image = hog(f_roi2[i], orientations=orn, pixels_per_cell=cs,
                        cells_per_block=(1, 1), visualize=True, multichannel=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(f_roi2[i], cmap=plt.cm.gray)
    ax1.set_title('Region of Interest')

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()

### Importing the Linear Kernel Of the Support Vector Machinr SVM

In [53]:
# import SVM
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
import joblib
import numpy as np

# Converting the labels from strings to integers
labels = [int(label) for label in labels]
#Converting the Data array to numpy array to be used by the SVM
data = np.array(data, dtype=np.float32)

# Partitioning the data into training and testing splits, using 90%
# of the data for training and the remaining 10% for testing
print(" Constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.10, random_state=42)


#%% Train the linear SVM
print(" Training Linear SVM classifier...")
model = LinearSVC() 
model.fit(trainData, trainLabels)
#%% Evaluate the classifier
print(" Evaluating classifier on test data ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))


# Save the model:
#%% Save the Model
joblib.dump(model, 'classification_model.npy')

 Constructing training/testing split...
 Training Linear SVM classifier...
 Evaluating classifier on test data ...
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       105
           1       0.96      0.92      0.94        96

    accuracy                           0.94       201
   macro avg       0.94      0.94      0.94       201
weighted avg       0.94      0.94      0.94       201



['classification_model.npy']

### Uploading and preprocessing new Images for Testing !!!!!

In [ ]:
import cv2
import glob
from PIL import Image
from resizeimage import resizeimage
import numpy as np

# Read RGB images from the validation folder
# Note put the images you want to test in the Validation Folder to be uploaded
Test = [cv2.imread(file) for file in glob.glob('Validation\\Test\\*.jpg')]
print("Done Uploading the Images")

#convert the images to greyscale
Test_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in Test]
print("Done Converting the Images to Gray")

#histogram equalization
Test_equlized = [cv2.equalizeHist(img) for img in Test_gray]

print("Apllied Histogram Equalization on the Gray Images")

#resizing
size = [90,90]
Test_resized = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in Test_equlized]
print("Done Resizing the Histogram Equalized Images to 90*90 Dimensions")

#Convert the resized arrays to numpy array to be used by the Haar Cascade !!
Test_sized= [np.array(img) for img in Test_resized]

# to view any of the images Whether colerd or Preprocessed Uncomment the next Lines !!!!
# AND Change the variable Named (Array_Name) to any of the preferred arrays above !!!!!!
#for img in Array_Name:
    # Output img with window name as 'image' 
    #cv2.imshow('image',img)
    # Maintain output window utill 
    # user presses a key 
    #cv2.waitKey(0)
    # Destroying present windows on screen 
    #cv2.destroyAllWindows()

#Upload the Haar Cascade Model for frontal Face Detection 
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Detect the Faces in the Test images
Test_haar= [face_cascade.detectMultiScale(
    img,
    scaleFactor=1.1,
    minNeighbors=3,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    ) for img in Test_sized]
    
print("Done Detecting the Test Faces")

   

# print The Number Of Detected Faces
print ("Found {0} faces!".format(len(Test_haar)))

# Extract the region of interest in the Test image (roi)
for f in Test_haar:
        for (x, y, w, h) in f:
            Test_roi = [img[y:y+h, x:x+w] for img in Test_sized] 
            
print("Extracted the region of interest for the Test faces")      


#resizing
size = [320,320]
Test_resized2 = [resizeimage.resize_cover(Image.fromarray(image), size, validate=False) for image in Test_roi]

Test_roi2 = [np.array(img) for img in Test_resizedt2]


In [ ]:
#To See the region of interest Extracted by the Haar Cascade
for img in Test_roi2:            
    cv2.imshow("Faces found", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

### Uploading the trained SVM Model for predicting the new test Images

In [59]:
# Upload the Saved svm model:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy import sparse

model = joblib.load('classification_model.npy') # Upload the Model !!!!!!!!!!!!


females_d = 0 # number of females detected
males_d = 0   # number of males detected

# settings for HOG
cs = (3, 3) #Cell Size
orn = 8     #Number of Orientations

# settings for LBP
radius = 3
n_points = 8 * radius


#Extract and predict whether the features belong to a Male or Female Face !!!!!!!!!!!!  
                        
for img in Test_roi2:
    fds = hog(img, orientations=orn, pixels_per_cell=cs, cells_per_block=(1, 1), block_norm='L2', feature_vector=True, multichannel=False)  # extract HOG features from the window captured
    hists = describe(img, n_points, radius)
    fds = fds.reshape(1, -1) # re shape the vector of HOG
    hists = hists.reshape(1, -1)    # re shape the vector of lBP
    feat = np.hstack([hists, fds])  #Combine the Features
    pred = model.predict(feat) # use the SVM model to make a prediction on the HOG and LBP features extracted from the Image

                     
    if pred == 1:
                if model.decision_function(feat) > 0.6:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                    females_d +=1
                    print("Detected A Female image")
                    print("Confidence Score {} \n".format(model.decision_function(feat)))

    if pred == 0:
                if model.decision_function(feat) < 0.6:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                    males_d +=1
                    print("Detected A Male image")
                    print("Confidence Score {} \n".format(model.decision_function(feat))) 
                    
print("Females Detected {} \n".format(females_d))
print("males Detected {} \n".format(males_d))
    

Detected A Male image
Confidence Score [-0.25408373] 

Detected A Male image
Confidence Score [-0.36962039] 

Detected A Male image
Confidence Score [-0.17271571] 

Detected A Male image
Confidence Score [-0.4938392] 

Detected A Male image
Confidence Score [-0.19372816] 

Detected A Male image
Confidence Score [-0.29279273] 

Detected A Male image
Confidence Score [-0.31216451] 

Detected A Male image
Confidence Score [-0.01558686] 

Detected A Male image
Confidence Score [-0.5804139] 

Detected A Male image
Confidence Score [-0.01112624] 

Detected A Male image
Confidence Score [-0.25273307] 

Detected A Male image
Confidence Score [-0.61678031] 

Detected A Male image
Confidence Score [-0.31463339] 

Detected A Male image
Confidence Score [-0.47541545] 

Detected A Male image
Confidence Score [-0.19856366] 

Detected A Male image
Confidence Score [-0.17314578] 

Detected A Male image
Confidence Score [-0.09684652] 

Detected A Male image
Confidence Score [-0.46026529] 

Detected A M